In [1]:
import sys
import folium
import geopandas as gpd

# path to folder to CityGeoTools
folder = "/var/essdata/IDU/other/Rest_/refactored/CityGeoTools"
sys.path.append(folder)

from metrics.data import CityInformationModel as BaseModel

/var/essdata/IDU/venvs/teaching/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [6]:
city_model = BaseModel.CityInformationModel(city_name="Saint-Petersburg", city_crs=32636, cwd="../")
city_model.update_layer("Services", "./data/Services.geojson")

Validation of Services layer...
Services layer loaded successfully!


In [7]:
city_model.methods.services_clusterization.message

{'Services': 'Layer matches specification'}

In [8]:
from metrics.calculations import services_clusterization

service_clusteriztion = services_clusterization.ServicesClusterization(city_model).get_clusters_polygon(
    service_types=['kindergartens', 'schools', 'cafes'], n_std=2, condition_value=1000
    )

service_poins = gpd.GeoDataFrame.from_features(service_clusteriztion['services']).set_crs(4326)
service_clusters = gpd.GeoDataFrame.from_features(service_clusteriztion['polygons']).set_crs(4326)

service_clusters.head(10)

,geometry,cafes,kindergartens,schools
0,"POLYGON ((30.61217 59.74231, 30.61092 59.74206...",0.00,1.00,0.00
1,"POLYGON ((30.60999 59.73162, 30.60875 59.73206...",0.33,0.33,0.33
2,"POLYGON ((30.61616 59.73741, 30.61532 59.73751...",0.00,0.60,0.40
3,"POLYGON ((30.58936 59.73697, 30.58854 59.73626...",0.20,0.60,0.20
4,"POLYGON ((30.58227 59.73110, 30.58142 59.73183...",0.00,0.50,0.50
5,"POLYGON ((30.56893 59.72906, 30.56799 59.72975...",0.20,0.80,0.00
6,"POLYGON ((30.58241 59.74801, 30.58185 59.74792...",0.33,0.33,0.33
7,"POLYGON ((30.58602 59.74184, 30.58517 59.74250...",0.00,0.75,0.25
8,"POLYGON ((30.56805 59.74875, 30.56675 59.74877...",0.33,0.67,0.00
9,"POLYGON ((30.57404 59.74402, 30.57305 59.74348...",0.10,0.50,0.40


In [9]:
service_poins.head(10)

,geometry,building_id,id,city_service_type,city_service_type_id,service_code,service_name,address,capacity,block_id,administrative_unit_id,municipality_id,x,y,cluster
0,POINT (30.31036 59.91364),61260.0,145852,Детский сад,1,kindergartens,Детский сад №151 (филиал),"Санкт-Петербург, 4-я Красноармейская улица, 20/8",82,166067,63,83,349614.919350,6.644848e+06,487
1,POINT (30.28720 59.96192),51977.0,145919,Детский сад,1,kindergartens,Детский сад №83,"Санкт-Петербург, Большая Разночинная улица, 25Б",280,165553,59,73,348540.878147,6.650275e+06,507
2,POINT (30.35694 59.85483),78106.0,145961,Детский сад,1,kindergartens,Детский сад №24,"Санкт-Петербург, Витебский проспект, 41к5",125,166788,57,57,351957.041670,6.638198e+06,134
3,POINT (30.23466 59.94038),37228.0,145973,Детский сад,1,kindergartens,Детский сад №5,"Санкт-Петербург, Гаванская улица, 56",170,164724,61,12,345508.065478,6.647999e+06,443
4,POINT (30.47480 59.88070),134136.0,146008,Детский сад,1,kindergartens,Детский сад №83,"Санкт-Петербург, Дальневосточный проспект, 66к2",166,168700,52,44,358666.079811,6.640820e+06,631
5,POINT (30.25425 59.84405),41535.0,146010,Детский сад,1,kindergartens,Детский сад №25,"Санкт-Петербург, Дачный проспект, 14к2",280,164651,51,38,346157.598749,6.637232e+06,183
6,POINT (30.24945 59.93165),40383.0,146020,Детский сад,1,kindergartens,ГБДОУ Детский сад №32,"Санкт-Петербург, Детская улица, 15",97,164717,61,12,346293.664318,6.646993e+06,450
7,POINT (30.40872 59.83410),92572.0,146043,Детский сад,1,kindergartens,Детский сад №44,"Санкт-Петербург, Загребский бульвар, 31к2",109,167795,62,82,354765.980946,6.635775e+06,145
8,POINT (30.56235 59.74106),113065.0,146119,Детский сад,1,kindergartens,Детский сад №6,"Санкт-Петербург, Колпино, Пролетарская улица, ...",220,168970,56,6,362992.654951,6.625093e+06,14
9,POINT (30.34491 59.94352),73834.0,146257,Детский сад,1,kindergartens,Детский сад №88,"Санкт-Петербург, Моховая улица, 19",77,166844,58,71,351679.084878,6.648096e+06,558


In [ ]:
# TODO: vizualization